## 回帰とは

* 機械学習の手法は、**教師あり学習** と **教師なし学習** に分類でき、  
更に教師あり学習は、 **回帰** と **分類** の問題に分けることができる。
* **回帰** は、入力に対して連続した数値を対応付ける問題で、  
ざっくり言うと入力値を元に、最適な関数を導き出して予測する手法のこと。

## 対象データ

* [DATA GO.JP](http://www.data.go.jp/?lang=japanese)にある  
**年齢別、1日の運動・スポーツ実施時間別体格測定・テストの結果**
 * 性別 男子
 * 実施状況 ほとんど毎日
 * テスト項目 上体起こし

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

ages = [6, 10, 15, 20, 25, 30]
times = [11.60, 17.86, 26.68, 27.01, 26.47, 25.10]

# |6|117.42|21.53|11.60|  
# |7|123.10|24.25|12.79|
# |8|128.57|26.76|14.71|
# |9|132.81|30.01|16.23|
# |10|138.83|34.13|17.86|  
# |11|144.61|38.32|18.09|
# |12|152.77|44.46|20.48|
# |13|158.35|48.88|24.56|
# |14|164.43|51.80|27.28|
# |15|167.10|56.58|26.68|
# |16|168.95|57.49|27.44|
# |17|170.15|59.51|29.59|
# |18|170.76|60.16|29.06|
# |19|171.56|61.78|29.08|
# |20|171.71|64.50|27.01|
# |25|171.86|67.17|26.47|
# |30|172.23|68.03|25.10|
# |35|171.79|69.04|23.35|
# |40|171.62|69.75|22.33|
# |45|171.12|69.74|21.51|
# |50|170.75|69.24|20.32|

plt.plot(ages, times, marker='o', linestyle='None')
plt.grid(True)
plt.xlabel('$age$')
plt.ylabel('$times$')
plt.show()